# P2WPKH(multisig) - BX Example

<br>
<img src="images/p2wsh.jpg" alt="drawing" style="" width="800px"/>

<hr style="border: 0.5px dashed #000;">




In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System.
#include <bitcoin/bitcoin.hpp> 


## 1. Spend to a P2WSH(multisig) output.


* Witness script: **`2/3 multsig`**
* Output script: **`zero [32-byte sha256(witness script)]`**

In [2]:
{
    //**************************************************************
    // 1. Restore wallet and generate spending / receiving key pairs
    //**************************************************************

    std::string my_sentence = "amount right cheese defy click eight slight strategy replace earn simple labor";
    const auto my_word_list = bc::split(my_sentence, " ", true);

    const auto hd_seed = bc::wallet::decode_mnemonic(my_word_list);

    bc::wallet::hd_private hd_master_private(bc::to_chunk(hd_seed),
        bc::wallet::hd_private::testnet);
    const auto hd_master_public = hd_master_private.to_public();

    // We will be spending a p2pkh output script controlled by
    // the following key: m/44'/1'/1'/0/9
    //-------------------------------------------------------------

    const auto hd_m_44h_1h_1h_0_9 = hd_master_private
        .derive_private(44 + bc::wallet::hd_first_hardened_key)
        .derive_private(1 + bc::wallet::hd_first_hardened_key)
        .derive_private(1 + bc::wallet::hd_first_hardened_key)
        .derive_private(0)
        .derive_private(9);
    const auto secret_44h_1h_1h_0_9 = hd_m_44h_1h_1h_0_9.secret();
    const auto pubkey_44h_1h_1h_0_9 =
        hd_m_44h_1h_1h_0_9.to_public().point();

    bc::wallet::payment_address payment_address_44h_1h_1h_0_9(
        pubkey_44h_1h_1h_0_9, bc::wallet::payment_address::testnet_p2kh);

    // Generate receiving keys for the P2SH(Multisig) output.
    //-------------------------------------------------------------

    // Three Receiving public keys in 3rd wallet account.

    const auto hd_m_44h_1h_2h_0 = hd_master_private
        .derive_private(44 + bc::wallet::hd_first_hardened_key)
        .derive_private(1 + bc::wallet::hd_first_hardened_key)
        .derive_private(2 + bc::wallet::hd_first_hardened_key)
        .derive_private(0);

    const auto hd_m_44h_1h_2h_0_9 = hd_m_44h_1h_2h_0.derive_private(9);
    const auto pubkey_44h_1h_2h_0_9 = hd_m_44h_1h_2h_0_9.to_public().point();

    const auto hd_m_44h_1h_2h_0_10 = hd_m_44h_1h_2h_0.derive_private(10);
    const auto pubkey_44h_1h_2h_0_10 = hd_m_44h_1h_2h_0_10.to_public().point();

    const auto hd_m_44h_1h_2h_0_11 = hd_m_44h_1h_2h_0.derive_private(11);
    const auto pubkey_44h_1h_2h_0_11 = hd_m_44h_1h_2h_0_11.to_public().point();

    //**************************************************************
    // 2. Fetch previous P2PKH UTXO which we will spend.
    //**************************************************************

    // Previous TX hash.
    const auto prev_tx_literal =
        "86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363";
    bc::hash_digest prev_tx_hash;
    bc::decode_hash(prev_tx_hash, prev_tx_literal);

    // Previous UXTO prev_index.
    uint32_t prev_index = 9;
    bc::chain::output_point uxto_to_spend(prev_tx_hash, prev_index);

    // Previous script.
    const auto prev_p2pkh_output_script =
        bc::chain::script::to_pay_key_hash_pattern(
            bc::bitcoin_short_hash(pubkey_44h_1h_1h_0_9));

    // Previous output amount.
    uint64_t previous_output_amount(499543);

    //**************************************************************
    // 3. Build, Sign & Complete Transaction.
    //**************************************************************

    // A. Encode P2SH(multisig) output.
    //-------------------------------------------------------------

    // Witness script: 2/3 Multisig.
    uint8_t signature_number(2u);
    bc::point_list points;
    points.push_back(pubkey_44h_1h_2h_0_9);
    points.push_back(pubkey_44h_1h_2h_0_10);
    points.push_back(pubkey_44h_1h_2h_0_11);
    bc::chain::script witness_script =
        bc::chain::script::to_pay_multisig_pattern(signature_number, points);

    // P2WSH output script.
    //    0 [32-byte sha256(witness script)]
    const auto witness_script_hash =
        bc::sha256_hash_chunk(witness_script.to_data(false)); // no prefix.
    bc::machine::operation::list p2sh_operations;
    p2sh_operations.push_back(
        bc::machine::operation(bc::machine::opcode::push_size_0));
    p2sh_operations.push_back(
        bc::machine::operation(witness_script_hash));

    // Compute fees.
    // tx bytes: 4 + 1 + 1*(32+4+110+4) + 1 + 1*(8+1+1+34) + 4 = 202
    // 1 non-witness sigop: 1 * 400
    // 499543 - 1 * 400 - 202 = 498941
    uint64_t output_amount(498941);

    // Build output.
    bc::chain::output p2wsh_output(output_amount, p2sh_operations);

    // B. Build Transaction for signing.
    //-------------------------------------------------------------

    // Build input0 object.
    bc::chain::input input0;
    input0.set_previous_output(uxto_to_spend);
    input0.set_sequence(bc::max_input_sequence);

    // Build Transaction.
    bc::chain::transaction tx;
    tx.set_version(1u);
    tx.inputs().push_back(input0);
    tx.outputs().push_back(p2wsh_output);

    // Signature.
    bc::endorsement signature;
    uint8_t input0_index(0u);
    bc::chain::script::create_endorsement(signature, secret_44h_1h_1h_0_9,
      prev_p2pkh_output_script, tx, input0_index,
      bc::machine::sighash_algorithm::all);

    // B. Finalise Transaction.
    //-------------------------------------------------------------

    // Set input script into transaction.
    bc::machine::operation::list p2pkh_operations;
    p2pkh_operations.push_back(bc::machine::operation(signature));
    p2pkh_operations.push_back(bc::machine::operation(
        bc::to_chunk(pubkey_44h_1h_1h_0_9)));
    bc::chain::script p2pkh_input_script(p2pkh_operations);
    tx.inputs()[0].set_script(p2pkh_input_script);

    std::cout << bc::encode_base16(tx.to_data()) << std::endl;
}

010000000163336e2138668a74127d5f9c8fab5862fa63645533cc33c2970492455663ad86090000006b48304502210085e59c38a5952ae54a7581720ebfb825a9632f12e0beb756bc074b3ee7c6bbbc0220282a9a230b294fa9ebe600e41a29945af11ebb8e4ee93afecd58f332f2de8d7d012102c8e0e676fd5d0c03e27604758e6c7450db599d4d43b9262c87dd5c95e57ff900ffffffff01fd9c070000000000220020a6bc7bb11b6583604df28a39250d70b4af6f0561332137fcaa265a38136c8c6000000000


In [13]:
!bx tx-decode -f json 010000000163336e2138668a74127d5f9c8fab5862fa63645533cc33c2970492455663ad86090000006b48304502210085e59c38a5952ae54a7581720ebfb825a9632f12e0beb756bc074b3ee7c6bbbc0220282a9a230b294fa9ebe600e41a29945af11ebb8e4ee93afecd58f332f2de8d7d012102c8e0e676fd5d0c03e27604758e6c7450db599d4d43b9262c87dd5c95e57ff900ffffffff01fd9c070000000000220020a6bc7bb11b6583604df28a39250d70b4af6f0561332137fcaa265a38136c8c6000000000 
    

{
    "transaction": {
        "hash": "2fc6c2abe0352b2cf1c4e1f48c10757e7a69b6b4f1a8252448a23221abbe00fa",
        "inputs": [
            {
                "address_hash": "af9cb1da5f66a5f129c1efa9970145c954d8a461",
                "previous_output": {
                    "hash": "86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363",
                    "index": "9"
                },
                "script": "[304502210085e59c38a5952ae54a7581720ebfb825a9632f12e0beb756bc074b3ee7c6bbbc0220282a9a230b294fa9ebe600e41a29945af11ebb8e4ee93afecd58f332f2de8d7d01] [02c8e0e676fd5d0c03e27604758e6c7450db599d4d43b9262c87dd5c95e57ff900]",
                "sequence": "4294967295"
            }
        ],
        "lock_time": "0",
        "outputs": [
            {
                "script": "zero [a6bc7bb11b6583604df28a39250d70b4af6f0561332137fcaa265a38136c8c60]",
                "value": "498941"
            }
        ],
        "version": "1"
    }
}


In [14]:
!bx validate-tx 010000000163336e2138668a74127d5f9c8fab5862fa63645533cc33c2970492455663ad86090000006b48304502210085e59c38a5952ae54a7581720ebfb825a9632f12e0beb756bc074b3ee7c6bbbc0220282a9a230b294fa9ebe600e41a29945af11ebb8e4ee93afecd58f332f2de8d7d012102c8e0e676fd5d0c03e27604758e6c7450db599d4d43b9262c87dd5c95e57ff900ffffffff01fd9c070000000000220020a6bc7bb11b6583604df28a39250d70b4af6f0561332137fcaa265a38136c8c6000000000
    

matching transaction with unspent outputs


<hr style="border: 0.5px dashed #000;">

## 2. Spend a P2WSH(multisig) output.

* Input script: **`"empty"`**
* Script code: **`witness script`**
* Witness: **`input script [witness script]`**

In [15]:
{
    //**************************************************************
    // 1. Restore wallet and generate spending key pairs
    //**************************************************************

    std::string my_sentence = "amount right cheese defy click eight slight strategy replace earn simple labor";
    const auto my_word_list = bc::split(my_sentence, " ", true);

    const auto hd_seed = bc::wallet::decode_mnemonic(my_word_list);

    bc::wallet::hd_private hd_master_private(bc::to_chunk(hd_seed),
        bc::wallet::hd_private::testnet);
    const auto hd_master_public = hd_master_private.to_public();

    // Three Receiving public keys in 3rd wallet account.
    //-------------------------------------------------------------

    const auto hd_m_44h_1h_2h_0 = hd_master_private
        .derive_private(44 + bc::wallet::hd_first_hardened_key)
        .derive_private(1 + bc::wallet::hd_first_hardened_key)
        .derive_private(2 + bc::wallet::hd_first_hardened_key)
        .derive_private(0);

    const auto hd_m_44h_1h_2h_0_9 = hd_m_44h_1h_2h_0.derive_private(9);
    const auto secret_44h_1h_2h_0_9 = hd_m_44h_1h_2h_0_9.secret();
    const auto pubkey_44h_1h_2h_0_9 = hd_m_44h_1h_2h_0_9.to_public().point();

    const auto hd_m_44h_1h_2h_0_10 = hd_m_44h_1h_2h_0.derive_private(10);
    const auto secret_44h_1h_2h_0_10 = hd_m_44h_1h_2h_0_10.secret();
    const auto pubkey_44h_1h_2h_0_10 = hd_m_44h_1h_2h_0_10.to_public().point();

    const auto hd_m_44h_1h_2h_0_11 = hd_m_44h_1h_2h_0.derive_private(11);
    const auto secret_44h_1h_2h_0_11 = hd_m_44h_1h_2h_0_11.secret();
    const auto pubkey_44h_1h_2h_0_11 = hd_m_44h_1h_2h_0_11.to_public().point();

    //**************************************************************
    // 2. Fetch previous P2PKH UTXO which we will spend.
    //**************************************************************

    // Previous TX hash.
    const auto prev_tx_literal =
        "2fc6c2abe0352b2cf1c4e1f48c10757e7a69b6b4f1a8252448a23221abbe00fa";
    bc::hash_digest prev_tx_hash;
    bc::decode_hash(prev_tx_hash, prev_tx_literal);

    // Previous UXTO prev_index.
    uint32_t prev_index = 0;
    bc::chain::output_point uxto_to_spend(prev_tx_hash, prev_index);

    // Previous output script not required for signing witness transactions.

    // Previous output amount.
    uint64_t previous_output_amount(498254);

    //**************************************************************
    // 3. Build, Sign & Complete Transaction.
    //**************************************************************

    // A. Encode P2PKH output.
    //-------------------------------------------------------------

    // We will send our funds to n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw
    // ...pubkeyhash: e48199d47742b245464b1366d95ef26aa4c8bb2c

    const auto receiving_pubkeyhash =
        bc::base16_literal("e48199d47742b245464b1366d95ef26aa4c8bb2c");
    const auto p2pkh_operations =
        bc::chain::script::to_pay_key_hash_pattern(receiving_pubkeyhash);

    // Compute fees.
    //-------------------------------------------------------------

    // tx bytes: 4 + 1 + 1*(32+4+0+4) + 1 + 1*(8+33) + 4 = 91

    // Witness tx size: (91) + 1 + 72 + 72 + 1 + 1 + 33 + 33 + 33 + 1 = 338

    // # of sigops: one multisig in script_code, one in output.
    // Output amount = 498951 - 1 * 20 * 100 - 1 * 400 - 338 * 1 = 496216

    uint64_t output_amount(496216);

    // Build P2WPKH output.
    bc::chain::output p2wpkh_output(output_amount, p2pkh_operations);

    // B. Build & Sign Transaction.
    //-------------------------------------------------------------

    // Build input0 object.
    bc::chain::input input;
    input.set_previous_output(uxto_to_spend);
    input.set_sequence(bc::max_input_sequence);

    // Build Transaction.
    bc::chain::transaction tx;
    tx.set_version(1);
    tx.inputs().push_back(input);
    tx.outputs().push_back(p2wpkh_output);

    // Script code: witness script = e.g. multisig
    uint8_t signatures(2); //2 of 3
    bc::point_list points;
    points.push_back(pubkey_44h_1h_2h_0_9);
    points.push_back(pubkey_44h_1h_2h_0_10);
    points.push_back(pubkey_44h_1h_2h_0_11);
    bc::chain::script witness_script =
        bc::chain::script::to_pay_multisig_pattern(signatures, points);

    // Previous input index.
    uint8_t input0_index(0);

    // Create signatures for witness.
    bc::endorsement sig0;
    bc::endorsement sig1;
    bc::chain::script::create_endorsement(sig0, secret_44h_1h_2h_0_9,
        witness_script, tx, input0_index, bc::machine::sighash_algorithm::all,
        bc::machine::script_version::zero, previous_output_amount);
    bc::chain::script::create_endorsement(sig1, secret_44h_1h_2h_0_10,
        witness_script, tx, input0_index, bc::machine::sighash_algorithm::all,
        bc::machine::script_version::zero, previous_output_amount);

    // Create witness.
    bc::data_stack witness_stack;
    bc::data_chunk empty_chunk;
    witness_stack.push_back(empty_chunk);
    witness_stack.push_back(sig0);
    witness_stack.push_back(sig1);
    witness_stack.push_back(witness_script.to_data(false));
    bc::chain::witness p2wsh_witness(witness_stack);
    tx.inputs()[0].set_witness(p2wsh_witness);

    // Serialisation to include witness: wire=true/witness=true
    std::cout << bc::encode_base16(tx.to_data(true, true)) << std::endl;
}

01000000000101fa00beab2132a2482425a8f1b4b6697a7e75108cf4e1c4f12c2b35e0abc2c62f0000000000ffffffff0158920700000000001976a914e48199d47742b245464b1366d95ef26aa4c8bb2c88ac04004730440220467018ac9dd7eb5d0d1ea658d0c00d1665f00f64a3dc5a0db806fd6ae108c17b02204c18ad2cb9e8b458b894b45c759366396882749953d75b984a81cd2347bd121101483045022100e873748971dff0a90cd9de8d283ece441f7ef42f3a8b123b810de08d3ba9e1d1022063561516aa29721cd9f958a11f2431511de15e23772033a2b2105561a92c8ec40169522102863e4d95089a6368c270649e15dd27ba138f1e23aec6bf150d3731bc7fc003802103fcc782abbe5c6c4eb2b55c8c3740245a6e39648dbfcb38abd0b1ca09b7f49b3121036d30c6fde9a93fec8167c0a8a555cef27f291168d2a92c0fd0ff075f0631165e53ae00000000


In [11]:
!bx tx-decode -f json 01000000000101fa00beab2132a2482425a8f1b4b6697a7e75108cf4e1c4f12c2b35e0abc2c62f0000000000ffffffff0158920700000000001976a914e48199d47742b245464b1366d95ef26aa4c8bb2c88ac04004730440220467018ac9dd7eb5d0d1ea658d0c00d1665f00f64a3dc5a0db806fd6ae108c17b02204c18ad2cb9e8b458b894b45c759366396882749953d75b984a81cd2347bd121101483045022100e873748971dff0a90cd9de8d283ece441f7ef42f3a8b123b810de08d3ba9e1d1022063561516aa29721cd9f958a11f2431511de15e23772033a2b2105561a92c8ec40169522102863e4d95089a6368c270649e15dd27ba138f1e23aec6bf150d3731bc7fc003802103fcc782abbe5c6c4eb2b55c8c3740245a6e39648dbfcb38abd0b1ca09b7f49b3121036d30c6fde9a93fec8167c0a8a555cef27f291168d2a92c0fd0ff075f0631165e53ae00000000
    

{
    "transaction": {
        "hash": "4d6b3b2cdb80970513f00fe0c6ee07e81d28b352d7f07758c193e3a5d3b0b8a6",
        "inputs": [
            {
                "previous_output": {
                    "hash": "2fc6c2abe0352b2cf1c4e1f48c10757e7a69b6b4f1a8252448a23221abbe00fa",
                    "index": "0"
                },
                "script": "",
                "sequence": "4294967295",
                "witness": "[] [30440220467018ac9dd7eb5d0d1ea658d0c00d1665f00f64a3dc5a0db806fd6ae108c17b02204c18ad2cb9e8b458b894b45c759366396882749953d75b984a81cd2347bd121101] [3045022100e873748971dff0a90cd9de8d283ece441f7ef42f3a8b123b810de08d3ba9e1d1022063561516aa29721cd9f958a11f2431511de15e23772033a2b2105561a92c8ec401] [522102863e4d95089a6368c270649e15dd27ba138f1e23aec6bf150d3731bc7fc003802103fcc782abbe5c6c4eb2b55c8c3740245a6e39648dbfcb38abd0b1ca09b7f49b3121036d30c6fde9a93fec8167c0a8a555cef27f291168d2a92c0fd0ff075f0631165e53ae]"
            }
        ],
        "lock_time": "0",
        "outpu

In [12]:
!bx validate-tx 01000000000101fa00beab2132a2482425a8f1b4b6697a7e75108cf4e1c4f12c2b35e0abc2c62f0000000000ffffffff0158920700000000001976a914e48199d47742b245464b1366d95ef26aa4c8bb2c88ac04004730440220467018ac9dd7eb5d0d1ea658d0c00d1665f00f64a3dc5a0db806fd6ae108c17b02204c18ad2cb9e8b458b894b45c759366396882749953d75b984a81cd2347bd121101483045022100e873748971dff0a90cd9de8d283ece441f7ef42f3a8b123b810de08d3ba9e1d1022063561516aa29721cd9f958a11f2431511de15e23772033a2b2105561a92c8ec40169522102863e4d95089a6368c270649e15dd27ba138f1e23aec6bf150d3731bc7fc003802103fcc782abbe5c6c4eb2b55c8c3740245a6e39648dbfcb38abd0b1ca09b7f49b3121036d30c6fde9a93fec8167c0a8a555cef27f291168d2a92c0fd0ff075f0631165e53ae00000000
    
    

stack false
